# **Estimación del precio para predios en Bogotá**

**Objetivo:** Este proyecto tiene como objetivo generar un modelo predictivo para el precio de los predios en la ciudad de Bogotá, con base a características estructurales y variables socioeconómicas asociadas a estos.​

**Fuentes de información:**​ Conjunto de predios nuevos que se han cargado a la plataforma de Finca Raiz​. Obtenidos mediante web scrapping.


## **1.Importación de las librerias**

In [30]:
!pip install --upgrade pandas-profiling

In [31]:
#Manejo de datos
import pandas as pd
import numpy as np

#Visualización de datos
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#Analisis profundo de datos
from ydata_profiling import ProfileReport

#Entrenamiento del modelo
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from joblib import dump

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## **2.Importación de los datos**

In [4]:
url = '/content/drive/MyDrive/MINE/Analisis_con_ML/Proyecto_final/output_fr.csv'
data = pd.read_csv(url)

In [ ]:
data.head()

,description,priceM2,isNew,livingArea,minLivingArea,maxLivingArea,address,products,id,frId,...,acceso-para-camiones,acceso-para-tractomulas,panoramica-un-lado,statusCode,finca-avicola,con-casa-prefabricada,local-exterior,soporte-de-gruas,tarima-de-descargas,casa-de-trabajadores
0,Es un proyecto de vivienda de interés social y...,5.999510e+06,True,0.0,25.48,31.24,Calle 139 94-55,[{'configuration': {'startDate': '2023-05-05T1...,f0fc46d2-7261-4033-8b13-731d9cdf0d1d,6792008.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VENTA APARTAESTUDIO ALEJANDRIA AZIMUT MF333 ...,6.340426e+06,False,62.0,0.00,0.00,CALLE 160 54 35,[{'configuration': {'startDate': '2022-10-21T0...,7f6dca69-f6ff-422e-8325-9d8757d9555d,7695732.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Apartamento con gran iluminación natural, tien...",3.524590e+06,False,57.0,0.00,0.00,Carrera 115 # 81 - 81,[{'configuration': {'startDate': '2023-03-16T0...,f20aa550-321b-4537-ab63-8ac5be508733,10005827.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Málaga es un conjunto residencial que está loc...,5.327435e+06,True,0.0,58.00,76.00,Carrera 89A BIS # 8A - 18,[],57e71fa0-c68d-4af9-803c-a02642664737,10080314.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"SE VENDE CASA APTA PARA OFICINAS, ZONA DE ...",7.485380e+03,False,171.0,0.00,0.00,KRA 14 79-14,[],97877fa0-1da8-4ce8-b6c7-992422f6cbc9,7976839.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **3.Tratamiento de datos**


### **3.1Partición de los datos de entrenamiento y prueba**

In [32]:
train, test = train_test_split(data, test_size=0.2, random_state=33)

### **3.2 Limpieza de datos atípicos**

In [33]:
def excluir_registros_atipicos(df, y_column):
    df_filtered = df.copy()
    df_filtered = df_filtered[(df_filtered[y_column] >= df_filtered[y_column].quantile(.25)-(df_filtered[y_column].quantile(.75)-df_filtered[y_column].quantile(.25))*1.5) &
                              (df_filtered[y_column] <= df_filtered[y_column].quantile(.75)+(df_filtered[y_column].quantile(.75)-df_filtered[y_column].quantile(.25))*1.5)]
    return df_filtered
train = excluir_registros_atipicos(train, 'price')
train = excluir_registros_atipicos(train, 'area')
train = excluir_registros_atipicos(train, 'administration.price')

### **3.2 Limpieza de datos columnas**

In [34]:
def eliminar_columnas_nulas(df, umbral):
    # Calcula el número máximo de valores nulos permitidos en una columna
    max_nulos = df.shape[0] * umbral
    # Obtiene el nombre de las columnas con exceso de valores nulos
    columnas_nulas = df.columns[df.isnull().sum() > max_nulos]
    # Elimina las columnas con exceso de valores nulos del DataFrame
    df = df.drop(columnas_nulas, axis=1)
    return df
train = eliminar_columnas_nulas(train,0.50 )

### **3.3 Eliminación de columnas con valores constantes**

In [35]:
def eliminar_columnas_constantes(df):
    columnas_constantes = []
    for columna in df.columns:
        if df[columna].nunique() == 1:
            columnas_constantes.append(columna)
    
    df = df.drop(columnas_constantes, axis=1)
    return df
train = eliminar_columnas_constantes(train)

### **3.4 Eliminación de columnas con alta cardinalidad**

In [36]:
def eliminar_columnas_alta_cardinalidad(df, umbral):
    columnas_alta_cardinalidad = []
    for columna in df.columns:
        if ((df[columna].nunique() > umbral) & (df[columna].dtype == 'object')):
            columnas_alta_cardinalidad.append(columna)    
    df = df.drop(columnas_alta_cardinalidad, axis=1)
    return df
train = eliminar_columnas_alta_cardinalidad(train, 6)

In [ ]:
train.shape

(51464, 52)

### **3.5 Selección de variables**

In [37]:
train = train[['administration.price','age.name','area','baths.id','client.type','condition.id','floor.id','garages.id','locations.lat','locations.lng','price','stratum.id']][train['price'].isna()==False]
test = test[['administration.price','age.name','area','baths.id','client.type','condition.id','floor.id','garages.id','locations.lat','locations.lng','price','stratum.id']][test['price'].isna()==False]      
train['stratum.id'] = train['stratum.id'].replace(100,1)
train['client.type'] = train['client.type'].replace('BUILDER','BROKER')
test['stratum.id'] = test['stratum.id'].replace(100,1)
test['client.type'] = test['client.type'].replace('BUILDER','BROKER')
test = excluir_registros_atipicos(test, 'price')
test = excluir_registros_atipicos(test, 'area')
test = excluir_registros_atipicos(test, 'administration.price')

### **3.6 Diccionario de datos**

**Campo**  | **Descripción** 
-------------------|------------------
administration.price     | Precio de la administración
age.name | Antiguedad 
area| Área del predio
baths.id | Cantidad de baños
client.type | Tipo de cliente
condition.id | Estado del predio
floor.id | Número de pisos
garages.id | Número de garages
locations.lat | Latitud del predio
locations.lng | Longitud del predio
price | Precio del predio
stratum.id | Estrato socioeconómico

### **3.7 Resúmen estadístico**

In [ ]:
ProfileReport(train)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

## **4.Evaluación de modelos - Regresión Lasso**

### **4.1 Entrenamiento del modelo**

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

X_train, y_train = train.drop(['price'],axis=1), train['price'] 

variables_categoricas = ['age.name','client.type']
variables_numericas = ['administration.price', 'area', 'baths.id', 'floor.id', 'garages.id', 'locations.lat', 'locations.lng', 'condition.id', 'stratum.id']
target = 'price'

preprocesador = ColumnTransformer(
    transformers=[
        ('codificacion', OneHotEncoder(), variables_categoricas),
        ('escalado', StandardScaler(), variables_numericas)
    ]
)

pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('lasso', Lasso())
])

parametros = {
    'lasso__alpha': [0.1, 1.0, 10.0]
}

grid_search = GridSearchCV(pipeline, parametros, cv=5)

In [44]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocesador',
                                        ColumnTransformer(transformers=[('codificacion',
                                                                         OneHotEncoder(),
                                                                         ['age.name',
                                                                          'client.type']),
                                                                        ('escalado',
                                                                         StandardScaler(),
                                                                         ['administration.price',
                                                                          'area',
                                                                          'baths.id',
                                                                          'floor.id',
                                                                          'garages.id',
                                                                          'locations.lat',
                                                                          'locations.lng',
                                                                          'condition.id',
                                                                          'stratum.id'])])),
                                       ('lasso', Lasso())]),
             param_grid={'lasso__alpha': [0.1, 1.0, 10.0]})

In [22]:
grid_search.best_params_

{'regresion__alpha': 100,
 'transformar__cat': OneHotEncoder(handle_unknown='ignore'),
 'transformar__num': MinMaxScaler()}

In [45]:
lasso_model = grid_search.best_estimator_['lasso']
feature_importance = lasso_model.coef_

In [46]:
best_model = grid_search.best_estimator_

# Obtener el preprocesador del pipeline
preprocesador = best_model.named_steps['preprocesador']

# Obtener las columnas después de aplicar el preprocesamiento (one-hot encoding)
columnas_categoricas = preprocesador.named_transformers_['codificacion'].get_feature_names_out(variables_categoricas)
columnas = np.concatenate([columnas_categoricas, variables_numericas])

# Obtener los coeficientes del modelo de regresión Lasso
lasso_coeficientes = best_model.named_steps['lasso'].coef_

# Crear un DataFrame para mostrar los coeficientes y su importancia
df_importancia = pd.DataFrame({'Característica': columnas, 'Importancia': lasso_coeficientes})

# Ordenar por valor absoluto de la importancia
df_importancia['Importancia Absoluta'] = np.abs(df_importancia['Importancia'])
df_importancia = df_importancia.sort_values(by='Importancia Absoluta', ascending=False)

# Imprimir la tabla de importancia de características
print(df_importancia[['Característica','Importancia Absoluta']].sort_values(by=['Importancia Absoluta']))

              Característica  Importancia Absoluta
7        client.type_PRIVATE          1.698348e-07
13             locations.lat          1.254334e+06
14             locations.lng          1.869189e+06
11                  floor.id          3.402262e+06
15              condition.id          3.539314e+06
0        age.name_1 a 8 años          4.461533e+06
12                garages.id          1.763455e+07
8       administration.price          3.527908e+07
6         client.type_BROKER          3.541864e+07
2       age.name_9 a 15 años          4.061529e+07
10                  baths.id          4.157257e+07
3     age.name_menor a 1 año          5.106615e+07
1      age.name_16 a 30 años          7.688865e+07
4    age.name_más de 30 años          8.849094e+07
16                stratum.id          1.278845e+08
9                       area          1.624343e+08
5   age.name_sin especificar          1.699754e+08


In [47]:
X_test, y_test = test.drop(['price'],axis=1), test['price'] 

y_train_pred = best_model.predict(train[variables_categoricas + variables_numericas])  # Predicciones en los datos de entrenamiento
y_test_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Métricas de desempeño para los datos de entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Métricas de desempeño para los datos de prueba
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
import math
# Imprimir las métricas de desempeño
print("Métricas de desempeño en los datos de entrenamiento:")
print("RMSE:", math.sqrt(mse_train))
print("MSE:", mse_train)
print("MAE:", mae_train)
print("R²:", r2_train)
print()
print("Métricas de desempeño en los datos de prueba:")
print("RMSE:", math.sqrt(mse_test))
print("MSE:", mse_test)
print("MAE:", mae_test)
print("R²:", r2_test)

Métricas de desempeño en los datos de entrenamiento:
RMSE: 183600584.58598563
MSE: 3.3709174660315664e+16
MAE: 125639792.74645269
R²: 0.7086547699559091

Métricas de desempeño en los datos de prueba:
RMSE: 176967020.60235557
MSE: 3.131732638087454e+16
MAE: 121170650.42092165
R²: 0.6988795784539038


## **5.Evaluación de modelos - Árbol de Regresión**

In [48]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

variables_categoricas = ['age.name','client.type']
variables_numericas = ['administration.price', 'area', 'baths.id', 'floor.id', 'garages.id', 'locations.lat', 'locations.lng', 'condition.id', 'stratum.id']
target = 'price'

preprocesador = ColumnTransformer(
    transformers=[
        ('codificacion', OneHotEncoder(), variables_categoricas),
        ('escalado', StandardScaler(), variables_numericas)
    ]
)

pipeline = Pipeline([
    ('preprocesador', preprocesador),
    ('arbol_regresion', DecisionTreeRegressor())
])

parametros = {
    'arbol_regresion__max_depth': [None, 5, 10],
    'arbol_regresion__min_samples_split': [2, 5, 10]
}

grid_search2 = GridSearchCV(pipeline, parametros, cv=5)
grid_search2.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocesador',
                                        ColumnTransformer(transformers=[('codificacion',
                                                                         OneHotEncoder(),
                                                                         ['age.name',
                                                                          'client.type']),
                                                                        ('escalado',
                                                                         StandardScaler(),
                                                                         ['administration.price',
                                                                          'area',
                                                                          'baths.id',
                                                                          'floor.id',
                                                                          'garages.id',
                                                                          'locations.lat',
                                                                          'locations.lng',
                                                                          'condition.id',
                                                                          'stratum.id'])])),
                                       ('arbol_regresion',
                                        DecisionTreeRegressor())]),
             param_grid={'arbol_regresion__max_depth': [None, 5, 10],
                         'arbol_regresion__min_samples_split': [2, 5, 10]})

In [49]:
grid_search2.best_params_

{'arbol_regresion__max_depth': 10, 'arbol_regresion__min_samples_split': 10}

In [51]:
arbol_regresion_model = grid_search2.best_estimator_['arbol_regresion']
feature_importance = arbol_regresion_model.feature_importances_

best_model = grid_search2.best_estimator_

# Obtener el preprocesador del pipeline
preprocesador = best_model.named_steps['preprocesador']


# Obtén las columnas después de aplicar el preprocesamiento (one-hot encoding)
columnas_categoricas = preprocesador.named_transformers_['codificacion'].get_feature_names_out(variables_categoricas)
columnas = np.concatenate([columnas_categoricas, variables_numericas])


# Crea un DataFrame para mostrar las características y su importancia
df_importancia = pd.DataFrame({'Característica': columnas, 'Importancia': feature_importance})

# Ordena por la importancia en orden descendente
df_importancia = df_importancia.sort_values(by='Importancia', ascending=False)

# Imprimir la tabla de importancia de características
#print(df_importancia[['Característica','Importancia Absoluta']].sort_values(by=['Importancia Absoluta']))

df_importancia

,Característica,Importancia
9,area,0.590851
16,stratum.id,0.253177
13,locations.lat,0.035368
12,garages.id,0.026245
14,locations.lng,0.025069
5,age.name_sin especificar,0.023385
0,age.name_1 a 8 años,0.013160
8,administration.price,0.011635
10,baths.id,0.008985
3,age.name_menor a 1 año,0.005952


In [52]:
y_train_pred = best_model.predict(train[variables_categoricas + variables_numericas])  # Predicciones en los datos de entrenamiento
y_test_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Métricas de desempeño para los datos de entrenamiento
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Métricas de desempeño para los datos de prueba
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)
import math
# Imprimir las métricas de desempeño
print("Métricas de desempeño en los datos de entrenamiento:")
print("RMSE:", math.sqrt(mse_train))
print("MSE:", mse_train)
print("MAE:", mae_train)
print("R²:", r2_train)
print()
print("Métricas de desempeño en los datos de prueba:")
print("RMSE:", math.sqrt(mse_test))
print("MSE:", mse_test)
print("MAE:", mae_test)
print("R²:", r2_test)

Métricas de desempeño en los datos de entrenamiento:
RMSE: 116902701.89967461
MSE: 1.3666241711444184e+16
MAE: 74809015.78256162
R²: 0.8818839566562805

Métricas de desempeño en los datos de prueba:
RMSE: 126003137.7887823
MSE: 1.5876790732618858e+16
MAE: 78782857.8838215
R²: 0.8473424627612851


## **6.Exportación de los modelos**

### **6.1 Regresión Lasso**

In [ ]:
dump(grid_search.best_estimator_, 'reg_lasso.joblib') 

['reg_lasso.joblib']

In [ ]:
from joblib import load

class PredictionModel:

    def __init__(self):
        self.model = load("reg_lasso.joblib")

    def make_predictions(self, data):
        result = self.model.predict(data)
        return result

### **6.2 Árbol de regresión**

In [ ]:
dump(grid_search2.best_estimator_, 'reg_arbol.joblib') 

['reg_arbol.joblib']

In [ ]:
from joblib import load

class PredictionModel:

    def __init__(self):
        self.model = load("reg_arbol.joblib")

    def make_predictions(self, data):
        result = self.model.predict(data)
        return result